# Электронный парамагнитный резонанс
### Бутаков Иван, Гладков Андрей; Б05-873

В работе изучается являение электронного парамагнитного резонанса. Наблюдается эффект резонансного поглощения электронами, находящимися на нижних энергетических уровнях в условиях воздействия внешнего магнитного поля.

В данной работе достижение резонанса осуществляется посредством регулировки силы создаваемого магнитного поля, а также частоты излучения. Измеряемой в эксперименте величиной является поглощаемая в образце мощность излучения (конкретнее, изучается зависимость мнимой частивысокочастотной восприимчивости от магнитного поля). Так как в эксперименте исследуется парамагнетик при комнатной температуре, далее будет неявно использоваться тот факт, что различия между индукцией и напряженностью магнитного поля в образце почти нет.

## Импортирование данных

In [1]:
import numpy

In [2]:
# Резонансная частота.
freq = 162.56e+06
freq_err = 0.05e+06

# Время между пиками.
tau = 6e-03
tau_err = 0.2e-03

#w = 2 * numpy.pi / tau
#w_err = tau_err / tau**2

# Угловая резонансная частота.
w = 2 * numpy.pi * freq
w_err = 2 * numpy.pi * freq_err

# Частота переменного тока.
w_AC = 2 * numpy.pi * 50.0
w_AC_err = 2 * numpy.pi * 0.2

In [3]:
# Падение напряжения на омическом сопротивлении.
U_R = 123.97
U_R_err = 0.05

# Падение напряжения на катушке при переменном токе.
U_L = 123.5
U_L_err = 0.5

# Положение ползунка на катушке.
V_L = 147.0
V_L_err = 2.0

In [4]:
# Результаты измерения поля магнетометром (часть первая).
U_s = 13.93e-03
U_s_err = 0.05e-03

# Результаты измерения поля магнетометром (часть первая).
U_s_2 = 3.95e-03
U_s_2_err = 0.05e-03

# Параметры пробной катушки магнетометра.
N_s = 45
d_s = 0.0152
d_s_err = 0.0001

In [5]:
# Ширина резонансного пика.
L = 7.2
L_err = 0.2

# Ширина пика на его полувысоте.
l = 4.0
l_err = 0.2

## Обработка

Без углубления в технические детали будем пользоваться тем, что снимаемое напряжение пропорционально эффективному сопротивлению возникающего ЭПР-эффекта; это достигается построением специального колебательного контура и достижением одновременно резонанса в нём и электронного парамагнитного резонанса, получая тем самым зависимость $\displaystyle \frac{\Delta U}{U} \approx \frac{2 \Delta Q}{Q} \approx \frac{2 \cdot R_{\text{ЭПР}}}{R_0} $.

### Измерение приложенного поля.

По результатам измерения напряжения на пробной катушке однозначно восстанавливается значение амплитуды малочастотного магнитного поля, действующего на образец. Данное поле выражается по следующей формуле:
$$
B = \frac{U}{N \cdot S \cdot \omega} = \frac{4 U}{N \cdot \pi d^2 \cdot \omega}
$$
Где $ U $ - напряжение на катушке, $ N $ - число витков, $\displaystyle S = \frac{\pi d^2}{4} $ - площадь сечения катушки, $ \omega $ - циклическая частота переменного поля.

В качестве $ \omega $ используется значение частоты переменного тока в сети. Согласно ГОСТ 29322-2014, $ \omega = 2 \pi \cdot (50 \pm 0.2) \; \text{с}^{-1} $.

In [6]:
def get_induction(U, N, d, w, U_err = 0.0, N_err = 0.0, d_err = 0.0, w_err = 0.0):
    B = 4.0 * U / (numpy.pi * w * N * d**2)
    B_err = B * numpy.sqrt((U_err / U)**2 + (N_err / N)**2 + 2.0 * (d_err / d)**2 + (w_err / w)**2)
    
    return B, B_err

In [7]:
B_0, B_0_err = get_induction(U_s, N_s, d_s, w_AC, U_s_err, 0.0, d_s_err, w_AC_err)
B_0_2, B_0_2_err = get_induction(U_s_2, N_s, d_s, w_AC, U_s_err, 0.0, d_s_err, w_AC_err)

In [8]:
print("Магнитная индукция, при которой достигается резонанс: (%.2e ± %.1e) Тл." % (B_0, B_0_err))

Магнитная индукция, при которой достигается резонанс: (5.43e-03 ± 5.8e-05) Тл.


### Получение g-фактора.

In [9]:
mu_B = 927.401e-26 # Магнетон Бора.
h_ = 1.054e-34     # Редуцированная постоянная Планка.

Зная параметры поля и излучения, при которых происходит резонанс, можно найти g-фактор электрона по следующей формуле:
$$
g = \frac{\hbar \omega_{res}}{\mu_{B} \cdot B}
$$
Где $ B $ - приложенное постоянное (или малочастотное) магнитное поле, $ \omega_{res} $ - циклическая резонансная частота излучения.

In [10]:
def get_g_factor(w, B, w_err = 0.0, B_err = 0.0):
    g = h_ * w / (mu_B * B)
    g_err = g * numpy.sqrt((B_err / B)**2 + (w_err / w)**2)
    
    return g, g_err

In [11]:
# g-фактор.
g, g_err = get_g_factor(w, B_0, w_err, B_0_err)
print("Полученная оценка g-фактора: (%.2f ± %.2f)" % (g, g_err))

Полученная оценка g-фактора: (2.14 ± 0.02)


Далее оценим достоверность полученного результата. Для этого исследуем картину резонансного пика на осциллографе и оценим его ширину на полувысоте. Оттуда получим оценку на точность определения $ B $ в случае резонанса.
$$
\Delta B = B \cdot \frac{2 l }{L}
$$
где $ l $ - ширина пика на полувысоте, $ L $ - ширина всего пика.

In [12]:
delta_B = B_0_2 * 2.0 * l / L
print("ΔB = %.2e Тл" % delta_B)

ΔB = 1.71e-03 Тл


Отсюда, условно используя $ \Delta B $ как ошибку определения g-фактора, получаем оценку на точность найденного значения.

In [13]:
_, g_final_err = get_g_factor(w, B_0, w_err, delta_B)
print("Допустимая вариация g-фактора: %.2f" % g_final_err)

Допустимая вариация g-фактора: 0.67


Видно, что данная вариация гораздо больше $ |g - g_{\text{ист}}| = 2.14 - 2 = 0.14 $. Значит, в рамках точности определения параметров резонанса, которую допускает наша далеко не идеальная установка (в веществе возможны спин-решёточные и спин-спиновые релаксации, также можно утверждать о неоднородности магнитного поля), мы получили предсказываемый теорией результат. Скорее всего, полученное значение даже ближе к эффективному g-фактору, чем к теоретическому значению, вычесленному для свободного электрона.

## Выводы

 - Получены значения частоты излучения и силы магнитного поля, при которых наблюдается резонанс.
 - Произведено визуальное наблюдение резонансного пика.
 - Получено значение g-фактора электронов в веществе.
 - По ширине резонансного пика на полувысоте произведена оценка достоверности полученного значения g-фактора. Показано, что в полученные ворота допуска предсказываемое теорией значение g-фактора гарантированно попадает.
 - Дано возможное объяснение большой ширины резонансного пика.